In [9]:
from collections import OrderedDict
import csv
import datetime

import requests
from pyquery import PyQuery as pq

In [10]:
# searchpage = "https://caseinfo.aoc.arkansas.gov/cconnect/PROD/public/ck_public_qry_doct.cp_dktrpt_new_case_report?backto=C&case_id=&begin_date=01/01/2010&end_date=11/09/2017&county_code=ALL&cort_code=ALL&locn_code=ALL&case_type=CD%20-%20CONDEMNATION/EMINENT%20DOMAIN&docket_code="
# r = requests.get(searchpage)
# searchhtml = r.content

# Yeah, this is having callback problems. OK. Moving on, here's a manual save:

In [11]:
with open('caseindex.html', "r") as caseindex:
    searchhtml = caseindex.read()

In [26]:
masterlist = []
for row in pq(searchhtml)("tr")[1:]:  # Skip first row, start with the real data
    rr = OrderedDict()
    rr['casenumber'] = pq(pq(row)("td")[1])("a").text()
    rr['dateraw'] = pq(row)("td")[0].text
    myyear = rr['dateraw'][-2:]
    if myyear > "50":
        myyear = "19" + myyear
    else:
        myyear = "20" + myyear
    myday = rr['dateraw'][:2]   
    mymonth = datetime.datetime.strptime(rr['dateraw'], '%d-%b-%y').month
    rr['dateclean'] = myyear + "-" + str(mymonth) + "-" + myday
    rr['caseurl'] = pq(pq(row)("td")[1])("a").attr('href')
    rr['casetitle'] = pq(pq(row)("td")[1]).contents()[1].strip()
    rr['casetype'] = pq(row)("td")[2].text.strip()
    rr['casejudge'] = pq(row)("td")[3].text
    rr['casecourt'] = pq(row)("td")[4].text.strip()
    masterlist.append(rr)   

In [15]:
keys = rr.keys()

In [16]:
with open('caseindexexport.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(keys)
    for rr in masterlist:
        writer.writerow(rr.values())